In [1]:
import sys
import copy
import torch
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
sys.path.append("/home/zanardi/Codes/ML/RONEK/ronek/")
from ronek2 import env
from ronek2 import const
from ronek2 import utils
from ronek2 import backend as bkd
from ronek2.systems import BoxAd
from ronek2.roms import CoBRAS

In [3]:
env_opts = {
  "backend": "numpy",
  "device": "cpu",
  "device_idx": 0,
  "nb_threads": 2,
  "epsilon": None,
  "floatx": "float64"
}
env.set(**env_opts)

In [4]:
path_to_dtb = "/home/zanardi/Codes/ML/RONEK/ronek/examples/CR_Ar/database"

In [5]:
system = BoxAd(
  species={k: path_to_dtb + "/species/" + k + ".json" for k in ("Ar", "Arp", "em")},
  kin_dtb=path_to_dtb + "/rates/kin_fit.p",
  use_coll_int_fit=True
)
system.compute_c_mat(max_mom=2)

In [6]:
rho = 1e-2
T = 3e4
Te = 3e2
mu_quad = {
  "x": np.array([rho, T, Te]).reshape(1,-1),
  "w": np.ones((1,1))
}

In [7]:
cobras = CoBRAS(
  system=system,
  tgrid={"start": 1e-12, "stop": 1e-3, "num": 25},
  mu_quad=mu_quad,
  path_to_saving="./",
  saving=True
)

In [ ]:
X, Y = cobras(
  nb_meas=3,
  xnot=[-2,-1],
  modes=False,
  pod=True
)

In [ ]:
X.shape, Y.shape

In [ ]:
cobras(
  nb_meas=4,
  xnot=[-2,-1],
  modes=True,
  pod=True
)